# Tensor Networks to Quantum Circuits
After running `python src/h6_sto3g_R1p2.py`, you generate integrals for the $H_6$ chains. Then you may run `julia src/h6_tn_dmrg.jl` for a DMRG run to obtain natural occupations, which are then found in `data/tn_no_from_dmrg.npz`.

We employ a minimal Qiskit pipeline that:
1. Loads TN outputs (`tn_no_from_dmrg.npz`)
2. Rotates PySCF integrals into TN natural orbital (NO) basis
3. Runs VQE(UCCSD) on the NO Hamiltonian
4. Compares convergence with a canonical-orbital baseline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---- load integrals from PySCF (saved earlier) ----
dat = np.load("../data/h6_sto3g_R1p2.npz", allow_pickle=True)
h1_mo = dat["h1"]          # (norb, norb)
eri_mo = dat["eri"]        # (norb, norb, norb, norb)
mo_e   = dat["mo_e"]       # MO orbital energies (for reference)

# ---- load TN natural orbitals ----
tn = np.load("../data/tn_no_from_dmrg.npz", allow_pickle=True)
U_no  = tn["U_no"]         # columns = NOs
no_occ = tn["no_occ"]
E_dmrg = float(tn["E_dmrg"])

norb = h1_mo.shape[0]
nelec = (3, 3)  # H6 -> 6 electrons total, spin singlet

print("NO occupations:", np.round(no_occ, 6))
print("DMRG energy (TN):", E_dmrg)

# ---- rotate integrals to NO basis ----
# h' = U^T h U ; (pq|rs)' = U^T_pα U_qβ U^T_rγ U_sδ (αβ|γδ)
h1_no = U_no.T @ h1_mo @ U_no
eri_no = np.einsum("pa,qb,rc,sd,abcd->pqrs", U_no, U_no, U_no, U_no, eri_mo, optimize=True)

# ---- Qiskit imports (tested with recent Qiskit Nature) ----
import qiskit, qiskit_algorithms, qiskit_nature
print("qiskit:", qiskit.__version__, "  qiskit-algorithms:", qiskit_algorithms.__version__, "  qiskit-nature:", qiskit_nature.__version__)
from qiskit.primitives import Estimator, BaseSampler  # should work after fixing versions
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.operators import FermionicOp
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.circuit.library.initial_states import HartreeFock


# --- helper to build qubit operator from raw integrals (chemist’s notation) ---
def qubit_op_from_integrals(h1, eri, nelec):
    num_spatial_orbitals = h1.shape[0]
    # ElectronicEnergy.from_raw_integrals expects chemist’s 2e
    ham = ElectronicEnergy.from_raw_integrals(h1, eri)
    problem = ElectronicStructureProblem(ham)
    second_q_op = problem.hamiltonian.second_q_op()
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(second_q_op)
    return qubit_op, problem, mapper, num_spatial_orbitals

# --- baseline: canonical orbital Hamiltonian ---
H_can, problem_can, mapper, norb_can = qubit_op_from_integrals(h1_mo, eri_mo, nelec)

# --- TN warm-start: NO Hamiltonian ---
H_no, problem_no, _, norb_no = qubit_op_from_integrals(h1_no, eri_no, nelec)

def make_vqe(qubit_op, problem, mapper, num_spatial_orbitals, nelec, init_point=None):
    # pass the tuple (n_alpha, n_beta), not sum(nelec)
    init_hf = HartreeFock(num_spatial_orbitals, nelec, mapper)
    ansatz = UCCSD(
        num_spatial_orbitals=num_spatial_orbitals,
        num_particles=nelec,                # <-- tuple here too
        qubit_mapper=mapper,
        initial_state=init_hf,
    )
    opt = L_BFGS_B(maxiter=200)
    est = Estimator()
    vqe = VQE(est, ansatz, opt, initial_point=init_point)
    res = vqe.compute_minimum_eigenvalue(qubit_op)
    hist = vqe.optimizer_history
    return res, hist


# ---- Run both and compare ----
res_can, hist_can = make_vqe(H_can, problem_can, mapper, norb_can, nelec, init_point=None)   # baseline
res_no,  hist_no  = make_vqe(H_no,  problem_no,  mapper, norb_no,  nelec, init_point=None)   # TN-NO warm-start

print("VQE (canonical)  =", res_can.optimal_value)
print("VQE (TN NO basis) =", res_no.optimal_value)

# ---- Plot convergence ----
def hist_to_curve(hist):
    # qiskit_algorithms stores 'fun' (best-so-far energy) and 'nfev' (eval count)
    x = hist.get("nfev", list(range(len(hist.get("fun", [])))))
    y = hist.get("fun", [])
    return x, y

xc, yc = hist_to_curve(hist_can)
xn, yn = hist_to_curve(hist_no)

plt.figure()
plt.plot(xc, yc, label="Canonical orbitals (random init)")
plt.plot(xn, yn, label="TN natural orbitals (HF init)")
plt.axhline(E_dmrg, ls="--", lw=1, label="DMRG (TN) reference")
plt.xlabel("Function evaluations")
plt.ylabel("Energy (Hartree)")
plt.legend()
plt.tight_layout()
plt.show()


NO occupations: [1.564875 1.042699 1.001075 0.999485 0.962071 0.429794]
DMRG energy (TN): -4.583302946652173
qiskit: 1.2.4   qiskit-algorithms: 0.3.1   qiskit-nature: 0.7.2


/tmp/ipykernel_18478/1770605873.py:69: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  est = Estimator()
